In [40]:
!pip install mysql-connector-python


Defaulting to user installation because normal site-packages is not writeable


In [4]:
import mysql.connector

# Step 1: Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='localhost',          # Host where MySQL server is running
    user='root',               #  MySQL username
    password='Snovvy682!',     #  MySQL password
    database='Amazon_db'       # The name of the database
)

# Step 2: Create a cursor to interact with the database
cursor = connection.cursor()

# Step 3: Function to call the stored procedure
def call_procedure_total_spending(start_date, end_date):
    """
    Calls the stored procedure 'TotalSpendingByDateRange' with the given start and end dates.
    Prints the total spending within the specified date range.
    """
    try:
        cursor.callproc('TotalSpendingByDateRange', [start_date, end_date])
        for result in cursor.stored_results():
            print(f"Total spending from {start_date} to {end_date}: {result.fetchall()[0][0]}")
    except mysql.connector.Error as err:
        print("Error calling procedure:", err)

# Step 4: Function to call the stored function
def call_function_count_products(category_id):
    """
    Calls the stored function 'CountProductsInCategory' with the given category ID.
    Prints the count of products in the specified category.
    """
    try:
        cursor.execute("SELECT CountProductsInCategory(%s)", (category_id,))
        result = cursor.fetchone()
        print(f"Total products in category ID {category_id}: {result[0]}")
    except mysql.connector.Error as err:
        print("Error calling function:", err)

# Step 5: Call the stored procedure with the chosen date range
# Test with a date range that has existing orders
call_procedure_total_spending('2024-10-22', '2024-10-23')

# Step 6: Call the stored function with the chosen category ID
call_function_count_products(2)  # Chosen category ID for testing

# Step 7: Close the cursor and the database connection
cursor.close()
connection.close()


Total spending from 2024-10-22 to 2024-10-23: 739.97
Total products in category ID 2: 1


In [5]:
import mysql.connector

# Step 1: Establish the database connection
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Snovvy682!',
    database='Amazon_db'
)
cursor = connection.cursor()

# Step 2: Define a function to execute queries with host variables
def execute_query(query, params):
    """
    Executes a query with placeholders and displays the results.
    """
    try:
        cursor.execute(query, params)
        results = cursor.fetchall()
        for row in results:
            print(row)
    except mysql.connector.Error as err:
        print("Error:", err)

# Step 3: Execute each query with dynamic parameters

# 1. Query to fetch products with price greater than a threshold
print("\nQuery 1: Products with price greater than 100")
execute_query("""
SELECT Product.name, Category.category_name
FROM Product
JOIN Category ON Product.category_id = Category.category_id
WHERE Product.price > %s;
""", (100,))

# 2. Query to fetch unique names from Customer and Product for a specific category
print("\nQuery 2: Unique names from Customer and Product in category ID 2")
execute_query("""
SELECT name FROM Customer
UNION
SELECT name FROM Product WHERE Product.category_id = %s;
""", (2,))

# 3. Query to calculate total spending by each customer since a given date
print("\nQuery 3: Total spent by each customer since 2024-01-01")
execute_query("""
SELECT Customer.name, SUM(`Order`.total_amount) AS total_spent
FROM Customer
JOIN `Order` ON Customer.customer_id = `Order`.customer_id
WHERE `Order`.order_date >= %s
GROUP BY Customer.name;
""", ('2024-01-01',))

# 4. Query to fetch products in a category name 'Electronics'
print("\nQuery 4: Products in category 'Electronics'")
execute_query("""
SELECT name
FROM Product
WHERE category_id IN (
    SELECT category_id FROM Category WHERE category_name = %s
);
""", ('Electronics',))

# 5. Query to find customers who have spent more than a certain amount
print("\nQuery 5: Customers with total spending above 500")
execute_query("""
SELECT name
FROM Customer
WHERE EXISTS (
    SELECT 1 FROM `Order`
    WHERE `Order`.customer_id = Customer.customer_id AND total_amount > %s
);
""", (500,))

# Step 4: Close the cursor and connection
cursor.close()
connection.close()



Query 1: Products with price greater than 100
('Smartphone', 'Electronics')

Query 2: Unique names from Customer and Product in category ID 2
('John Doe',)
('Jane Smith',)
('T-shirt',)

Query 3: Total spent by each customer since 2024-01-01
('John Doe', Decimal('719.98'))
('Jane Smith', Decimal('19.99'))

Query 4: Products in category 'Electronics'
('Smartphone',)

Query 5: Customers with total spending above 500
('John Doe',)


In [6]:
import mysql.connector

# Define a function to connect to the database
def create_connection():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='Snovvy682!',
        database='Amazon_db'
    )

# Call the stored procedure for total spending within a date range
def call_procedure_total_spending(start_date, end_date):
    """
    Calls the stored procedure 'TotalSpendingByDateRange' with the given start and end dates.
    Prints the total spending within the specified date range.
    """
    # Establish a new connection and cursor
    connection = create_connection()
    cursor = connection.cursor()
    try:
        cursor.callproc('TotalSpendingByDateRange', [start_date, end_date])
        for result in cursor.stored_results():
            print(f"Total spending from {start_date} to {end_date}: {result.fetchall()[0][0]}")
    except mysql.connector.Error as err:
        print("Error calling procedure:", err)
    finally:
        # Ensure the cursor and connection are closed
        cursor.close()
        connection.close()

# Call the stored function for counting products in a specific category
def call_function_count_products(category_id):
    """
    Calls the stored function 'CountProductsInCategory' with the given category ID.
    Prints the count of products in the specified category.
    """
    # Establish a new connection and cursor
    connection = create_connection()
    cursor = connection.cursor()
    try:
        cursor.execute("SELECT CountProductsInCategory(%s)", (category_id,))
        result = cursor.fetchone()
        print(f"Total products in category ID {category_id}: {result[0]}")
    except mysql.connector.Error as err:
        print("Error calling function:", err)
    finally:
        # Ensure the cursor and connection are closed
        cursor.close()
        connection.close()

# Run the procedure and function with example values
# 1. Total spending within a specific date range
call_procedure_total_spending('2024-10-22', '2024-10-23')

# 2. Count of products in a specific category
call_function_count_products(2)  # Example with category ID 2


Total spending from 2024-10-22 to 2024-10-23: 739.97
Total products in category ID 2: 1


In [7]:
import mysql.connector

# Define a function to create a database connection and print connection status
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='Snovvy682!',
            database='Amazon_db'
        )
        print("Database connection established.")
        return connection
    except mysql.connector.Error as err:
        print("Error connecting to database:", err)
        return None

# Function to add an item to the cart using the stored procedure
def add_to_cart(customer_id, product_id, quantity):
    connection = create_connection()
    if connection is None:
        print("Failed to establish database connection.")
        return
    
    cursor = connection.cursor()
    try:
        # Call the stored procedure to add item to cart
        cursor.callproc('AddToCart', [customer_id, product_id, quantity])
        connection.commit()
        print(f"Added product {product_id} to cart with quantity {quantity}.")
    except mysql.connector.Error as err:
        print("Error adding to cart:", err)
    finally:
        cursor.close()
        connection.close()
        print("Connection closed after adding to cart.")

# Function to calculate the total cost of items in the cart using the stored function
def calculate_cart_total(customer_id):
    connection = create_connection()
    if connection is None:
        print("Failed to establish database connection.")
        return
    
    cursor = connection.cursor()
    try:
        # Call the stored function to calculate cart total
        cursor.execute("SELECT CalculateCartTotal(%s)", (customer_id,))
        result = cursor.fetchone()
        if result and result[0] is not None:
            print(f"Total cart cost for customer {customer_id}: ${result[0]}")
        else:
            print(f"No items in cart for customer {customer_id} or total is zero.")
    except mysql.connector.Error as err:
        print("Error calculating cart total:", err)
    finally:
        cursor.close()
        connection.close()
        print("Connection closed after calculating cart total.")

# Example usage
print("Adding items to the cart...")
add_to_cart(1, 101, 2)  # Add 2 units of product 101 for customer 1
add_to_cart(1, 102, 1)  # Add 1 unit of product 102 for customer 1

print("\nCalculating cart total...")
calculate_cart_total(1)


Adding items to the cart...
Database connection established.
Added product 101 to cart with quantity 2.
Connection closed after adding to cart.
Database connection established.
Added product 102 to cart with quantity 1.
Connection closed after adding to cart.

Calculating cart total...
Database connection established.
Total cart cost for customer 1: $1399.98
Connection closed after calculating cart total.


In [8]:
# Adding items to customer 1's cart with existing product IDs
add_to_cart(1, 1, 2)  # Adds 2 units of product 1 (Smartphone) to customer 1's cart
add_to_cart(1, 2, 1)  # Adds 1 unit of product 2 (T-shirt) to customer 1's cart


Database connection established.
Added product 1 to cart with quantity 2.
Connection closed after adding to cart.
Database connection established.
Added product 2 to cart with quantity 1.
Connection closed after adding to cart.


In [9]:
# Calculate the total cost of items in customer 1's cart
calculate_cart_total(1)


Database connection established.
Total cart cost for customer 1: $1399.98
Connection closed after calculating cart total.


In [10]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector

# Function to create a database connection
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='Snovvy682!',
            database='Amazon_db'
        )
        print("Database connection established.")
        return connection
    except mysql.connector.Error as err:
        print("Error connecting to database:", err)
        return None

# Function to add an item to the cart
def add_to_cart():
    try:
        customer_id = int(customer_id_entry.get())
        product_id = int(product_id_entry.get())
        quantity = int(quantity_entry.get())
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numbers for Customer ID, Product ID, and Quantity.")
        return
    
    connection = create_connection()
    if connection is None:
        print("Failed to establish database connection.")
        return
    
    cursor = connection.cursor()
    try:
        cursor.callproc('AddToCart', [customer_id, product_id, quantity])
        connection.commit()
        messagebox.showinfo("Success", f"Added product {product_id} to cart with quantity {quantity}.")
    except mysql.connector.Error as err:
        print("Error adding to cart:", err)
        messagebox.showerror("Error", f"Error adding to cart: {err}")
    finally:
        cursor.close()
        connection.close()
        print("Connection closed after adding to cart.")

# Function to calculate the total cost of items in the cart
def calculate_cart_total():
    try:
        customer_id = int(cart_customer_id_entry.get())
    except ValueError:
        messagebox.showerror("Input Error", "Please enter a valid number for Customer ID.")
        return
    
    connection = create_connection()
    if connection is None:
        print("Failed to establish database connection.")
        return
    
    cursor = connection.cursor()
    try:
        cursor.execute("SELECT CalculateCartTotal(%s)", (customer_id,))
        result = cursor.fetchone()
        if result and result[0] is not None:
            messagebox.showinfo("Cart Total", f"Total cart cost for customer {customer_id}: ${result[0]}")
        else:
            messagebox.showinfo("Cart Total", f"No items in cart for customer {customer_id} or total is zero.")
    except mysql.connector.Error as err:
        print("Error calculating cart total:", err)
        messagebox.showerror("Error", f"Error calculating cart total: {err}")
    finally:
        cursor.close()
        connection.close()
        print("Connection closed after calculating cart total.")

# Set up the main window with a white background and distinct colors
root = tk.Tk()
root.title("Cart Management GUI")
root.configure(bg="white")  # Set the background color to white

# Section for adding an item to the cart
tk.Label(root, text="Add Item to Cart", bg="white", fg="black", font=("Arial", 12, "bold")).grid(row=0, column=0, columnspan=2, pady=10)
tk.Label(root, text="Customer ID:", bg="white", fg="black", font=("Arial", 10, "bold")).grid(row=1, column=0, padx=5, pady=5, sticky="e")
customer_id_entry = tk.Entry(root, width=20, bg="lightblue", fg="black")
customer_id_entry.grid(row=1, column=1, padx=5, pady=5)

tk.Label(root, text="Product ID:", bg="white", fg="black", font=("Arial", 10, "bold")).grid(row=2, column=0, padx=5, pady=5, sticky="e")
product_id_entry = tk.Entry(root, width=20, bg="lightgreen", fg="black")
product_id_entry.grid(row=2, column=1, padx=5, pady=5)

tk.Label(root, text="Quantity:", bg="white", fg="black", font=("Arial", 10, "bold")).grid(row=3, column=0, padx=5, pady=5, sticky="e")
quantity_entry = tk.Entry(root, width=20, bg="lightpink", fg="black")
quantity_entry.grid(row=3, column=1, padx=5, pady=5)

add_button = tk.Button(root, text="Add to Cart", command=add_to_cart, bg="blue", fg="white", font=("Arial", 10, "bold"))
add_button.grid(row=4, column=0, columnspan=2, pady=10)

# Section for calculating the cart total
tk.Label(root, text="Calculate Cart Total", bg="white", fg="black", font=("Arial", 12, "bold")).grid(row=5, column=0, columnspan=2, pady=10)
tk.Label(root, text="Customer ID:", bg="white", fg="black", font=("Arial", 10, "bold")).grid(row=6, column=0, padx=5, pady=5, sticky="e")
cart_customer_id_entry = tk.Entry(root, width=20, bg="lightyellow", fg="black")
cart_customer_id_entry.grid(row=6, column=1, padx=5, pady=5)

calculate_button = tk.Button(root, text="Calculate Total", command=calculate_cart_total, bg="green", fg="white", font=("Arial", 10, "bold"))
calculate_button.grid(row=7, column=0, columnspan=2, pady=10)

# Start the GUI event loop
root.mainloop()